In [ ]:
#!pip install streamlit

In [ ]:
# Utilities
import matplotlib.pyplot as plt
import pandas as pd
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

## Credentials

In [ ]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

## Step 1: Define the Region of Interest (ROI)

In [ ]:
# Define the region of interest (ROI)
import geopandas as gpd

# Load the shapefile for the Midwest Corn Belt (Iowa/Illinois)
corn_belt_shapefile = 'path/to/corn_belt_shapefile.shp'  # Replace with actual path
corn_belt_roi = gpd.read_file(corn_belt_shapefile)

# Display the ROI
corn_belt_roi.plot()

## Step 2: Collect and Preprocess Datasets
1. Download Sentinel-2 NDVI Data

In [ ]:
from sentinelhub import SHConfig, BBox, CRS, MimeType, SentinelHubRequest, DataCollection
import numpy as np
import rasterio

# Define your Sentinel Hub configuration
#config = SHConfig()

# Define the bounding box for the Corn Belt (Iowa/Illinois)
corn_belt_bbox = BBox(bbox=[-91.5, 40.0, -88.0, 42.5], crs=CRS.WGS84)

# Define the evalscript for NDVI calculation
evalscript_ndvi = """
//VERSION=3
function setup() {
  return {
    input: ["B04", "B08"],
    output: { bands: 1 }
  };
}

function evaluatePixel(sample) {
  let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
  return [ndvi];
}
"""

# Create a request for NDVI data using Sentinel-2 L2A
ndvi_request = SentinelHubRequest(
    evalscript=evalscript_ndvi,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            time_interval=('2022-04-01', '2022-10-01')
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.TIFF)
    ],
    bbox=corn_belt_bbox,
    size=(512, 512),
    config=config
)

# Execute the request
ndvi_data = ndvi_request.get_data()

print("NDVI data has been retrieved successfully.")

2. Download Sentinel-1 Soil Moisture Estimates

In [ ]:
# Create a request for Soil Moisture data from Sentinel-1
soil_moisture_request = SentinelHubRequest(
    data_folder='Soil_Moisture_Data',
    data_source='S1GRD',
    bbox=corn_belt_bbox,
    time=('2022-04-01', '2022-10-01'),
    width=512,
    height=512,
    responses=[
        SentinelHubRequest.output_response('Soil_Moisture', MimeType.TIFF)
    ],
    config=config
)

# Execute the request
soil_moisture_data = soil_moisture_request.get_data()


3. Download GPM Rainfall Data

In [ ]:
import xarray as xr

# Download GPM rainfall data for the specific period
gpm_url = 'URL_TO_GPM_DATA'  # Replace with actual GPM data URL
gpm_data = xr.open_dataset(gpm_url)

# Select relevant time and region
rainfall_data = gpm_data.sel(time=slice('2022-04-01', '2022-10-01'))


4. Download MODIS Land Surface Temperature

In [ ]:
modis_url = 'URL_TO_MODIS_LST_DATA'  # Replace with actual MODIS data URL
lst_data = xr.open_dataset(modis_url)

# Select relevant time and region
temperature_data = lst_data.sel(time=slice('2022-04-01', '2022-10-01'))


## Step 3: Water Stress Assessment

In [ ]:
# Calculate NDVI anomalies
ndvi_mean = np.mean(ndvi_data)
ndvi_anomalies = ndvi_data - ndvi_mean

# Overlay rainfall and soil moisture data
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(ndvi_anomalies, label='NDVI Anomalies')
plt.title('NDVI Anomalies')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(rainfall_data, label='Rainfall Data', color='orange')
plt.title('Rainfall Data')
plt.legend()

plt.tight_layout()
plt.show()


### Step 4: Basic Crop Yield Prediction Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Historical data (assuming you have this data available)
historical_data = {
    'NDVI': [0.6, 0.65, 0.7, 0.75, 0.72],
    'Rainfall': [55, 60, 45, 30, 35],  # mm
    'SoilMoisture': [0.2, 0.25, 0.22, 0.18, 0.2],  # m³/m³
    'Yield': [3000, 3200, 3400, 3600, 3500]  # kg/ha
}

df = pd.DataFrame(historical_data)

# Train-test split
X = df[['NDVI', 'Rainfall', 'SoilMoisture']]
y = df['Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict yield based on a sample input
predicted_yield = model.predict([[0.76, 50, 0.25]])  # Replace with forecasted values
print(f"Predicted Crop Yield: {predicted_yield[0]:.2f} kg/ha")


### Step 5: Dashboard Interface

In [ ]:
import streamlit as st

# Dashboard title
st.title('Crop Health and Yield Prediction Dashboard')

# Display NDVI Time Series
st.subheader('NDVI Time Series')
st.line_chart(ndvi_data)

# Display rainfall data
st.subheader('Rainfall Data')
st.line_chart(rainfall_data)

# User input for predictions
st.subheader('Predict Crop Yield')
ndvi_input = st.number_input('Enter NDVI:', value=0.76)
rainfall_input = st.number_input('Enter Rainfall (mm):', value=50)
soil_moisture_input = st.number_input('Enter Soil Moisture (m³/m³):', value=0.25)

# Predict button
if st.button('Predict Yield'):
    input_features = np.array([[ndvi_input, rainfall_input, soil_moisture_input]])
    predicted_yield_dashboard = model.predict(input_features)
    st.write(f"Predicted Crop Yield: {predicted_yield_dashboard[0]:.2f} kg/ha")
